# Neural Distinguisher of Reduced Round Speck 32/64

## Speck 32/64

Speck 32/64의 구현이 맞는지 테스트벡터를 우선 확인해보겠습니다.

In [1]:
from speck import Speck

cipher = Speck()
cipher.check_testvector()

Testvector verified.


True

## Neural Distinguisher

Speck32/64의 5, 6, 7라운드 distinguisher의 모델을 생성하고 학습시켜보겠습니다. 

사용한 모델은 다음과 같습니다.자세한 코드는 train.py를 참고하면 됩니다.

- 은닉계층이 2개인 DNN
- Residual Block이 하나인 ResNet
- Residual Block이 10개인 ResNet

시간관계상 epoch를 1로 설정했습니다. 

매 epoch를 진행하면서 validation accuracy 가 가장 높은 상태를 .h5 확장자 파일로 저장합니다.

In [2]:
import train

train.train_all(epochs=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________
Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 17s 2us/sample - loss: 0

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 41s 4us/sample - loss: 0.0858 - acc: 0.8963 - val_loss: 0.0688 - val_acc: 0.9175
Best validaation accuracy:  0.91751
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 4, 16)        0           input_5[0][0]                    
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 16, 4)        0           reshape_1[0][0]                  
_________________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 43s 4us/sample - loss: 0.2520 - acc: 0.5223 - val_loss: 0.2446 - val_acc: 0.5690
Best validaation accuracy:  0.569047
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 4, 16)        0           input_7[0][0]                    
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 16, 4)        0           reshape_3[0][0]                  
________________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 168s 17us/sample - loss: 0.0816 - acc: 0.9043 - val_loss: 0.0668 - val_acc: 0.9203
Best validaation accuracy:  0.920284
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_4 (Reshape)             (None, 4, 16)        0           input_8[0][0]                    
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 16, 4)        0           reshape_4[0][0]                  
______________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 172s 17us/sample - loss: 0.1773 - acc: 0.7476 - val_loss: 0.1633 - val_acc: 0.7689
Best validaation accuracy:  0.768877
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 4, 16)        0           input_9[0][0]                    
__________________________________________________________________________________________________
permute_5 (Permute)             (None, 16, 4)        0           reshape_5[0][0]                  
______________________________________________________________

Train on 10000000 samples, validate on 1000000 samples
10000000/10000000 [==============================] - 176s 18us/sample - loss: 0.2532 - acc: 0.5272 - val_loss: 0.2424 - val_acc: 0.5864
Best validaation accuracy:  0.586351


epoch이 1이기 때문에 성능이 논문에 적힌 것보다 좋게 나오지 않습니다.

제가 epoch을 200으로 하여 학습시킨 데이터를 trained_models 폴더에 옮겨두었습니다.

해당 모델을 불러들여서 테스트 하는 코드가 evaluate.py에 구현되어 있습니다.

In [7]:
import evaluate

In [4]:
#DNN 결과
evaluate.evaluate_dnn()

Round:  5  Accuracy:  0.883779
Round:  6  Accuracy:  0.692097
Round:  7  Accuracy:  0.521133
Round:  8  Accuracy:  0.50102


In [6]:
#ResNet (block=1) 결과
evaluate.evaluate_resnet(resnet_depth=1)

Round:  5  Accuracy:  0.926386
Round:  6  Accuracy:  0.784676
Round:  7  Accuracy:  0.60734
Round:  8  Accuracy:  0.50019


In [8]:
#ResNet (block=10) 결과
evaluate.evaluate_resnet(resnet_depth=10)

Round:  5  Accuracy:  0.926835
Round:  6  Accuracy:  0.786839
Round:  7  Accuracy:  0.609867
Round:  8  Accuracy:  0.499761


## Key Recovery Attack

5라운드 distinguisher를 사용하여 6라운드 Speck 32/64의 키를 복구해보겠습니다.

해당 코드는 key_recovery.py 에 구현되어 있습니다.

In [1]:
import key_recovery

key_recovery.attack(num_test=5, target_round=6)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 4, 16)        0           input_1[0][0]                    
__________________________________________________________________________________________________
permute (Permute)               (None, 16, 4)        0           reshape[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 16, 32)       160         permute[0]

D:\workspace\git\neural_distinguisher\key_recovery.py:29: RuntimeWarning: divide by zero encountered in log2
  v = np.log2(v)


Recovery attack ended:  2019-09-01 17:58:18.356341
Elapsed 170.5656385421753
Guessed 0xda87 , Rank 622.3093872070312
Guessed 0x5a87 , Rank 619.9765625
Guessed 0x1a87 , Rank 612.9024047851562
Guessed 0x9a87 , Rank 610.9473266601562
Real key 0x5a87 , Rank 619.97656

<<Test 1 >>
Recovery attack started:  2019-09-01 17:58:18.363344
Recovery attack ended:  2019-09-01 18:01:10.685958
Elapsed 172.32261395454407
Guessed 0xb76a , Rank 548.8448486328125
Guessed 0x376a , Rank 546.8692626953125
Guessed 0xf76a , Rank 539.438720703125
Guessed 0x776a , Rank 539.3494873046875
Real key 0xb76a , Rank 548.84485

<<Test 2 >>
Recovery attack started:  2019-09-01 18:01:10.691960
Recovery attack ended:  2019-09-01 18:04:05.001395
Elapsed 174.30943536758423
Guessed 0xe9ec , Rank 564.3479614257812
Guessed 0x69ec , Rank 563.1509399414062
Guessed 0x29ec , Rank 558.1258544921875
Guessed 0xa9ec , Rank 557.0463256835938
Real key 0x69ec , Rank 563.15094

<<Test 3 >>
Recovery attack started:  2019-09-01 18:04:05.0073